<a href="https://colab.research.google.com/github/Ali-mohammadi-design/Prompt_Engineering_and_Machine_Learning/blob/main/Prompt_engineering_LangChain_Vector_Store_Multi_query_retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Sometimes the documents in your vector store may contain phrasing that you are not aware of due to their size. This can cause issues in trying to think of the correct query string for similarity comparisons. We can use an LLM to generate multiple variations of our query Using Multiquery retriever allowing us to focus on key ideas rather than exact phrasing**

In other words, multi_query_retirevers also retrieve the phrasing which are similar to the first phrase

In [1]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.6/105.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 25.

In [2]:
!pip install langchain
!pip install openai
!pip install wikipedia
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.6/229.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.4/223.4 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 5.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
  Preparing metadata (setup.py) ... done
  Created wheel for wikiped

In [3]:
import chromadb

In [4]:
from langchain.embeddings import OpenAIEmbeddings

In [5]:
from langchain.text_splitter import CharacterTextSplitter

In [6]:
from langchain.vectorstores import Chroma

In [7]:
from langchain.document_loaders import TextLoader, WikipediaLoader

In [9]:
loader=WikipediaLoader(query='MKULTRA')
documents=loader.load()

/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [10]:
len(documents)

24

As is clear there are lots of document here. We can break it up to some smaller chunks

In [13]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 7.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [12]:
from langchain.text_splitter import CharacterTextSplitter

In [17]:
text_splitter= CharacterTextSplitter.from_tiktoken_encoder(chunk_size=500)

In [19]:
docs=text_splitter.split_documents(documents)

In [20]:
docs[0]

Document(page_content="Project MKUltra was an illegal human experimentation program designed and undertaken by the U.S. Central Intelligence Agency (CIA) and intended to develop procedures and identify drugs that could be used during interrogations to weaken people and force confessions through brainwashing and psychological torture. It began in 1953 and was halted in 1973. MKUltra used numerous methods to manipulate its subjects' mental states and brain functions, such as the covert administration of high doses of psychoactive drugs (especially LSD) and other chemicals without the subjects' consent, electroshocks, hypnosis, sensory deprivation, isolation, verbal and sexual abuse, and other forms of torture.MKUltra was preceded by Project Artichoke. It was organized through the CIA's Office of Scientific Intelligence and coordinated with the United States Army Biological Warfare Laboratories. The program engaged in illegal activities, including the use of U.S. and Canadian citizens as 

In [21]:
len(docs)

51

In [23]:
from langchain.embeddings import OpenAIEmbeddings

In [25]:
import os
openAI_API_Key=os.environ['openAI_API_Key']

In [27]:
my_embedding_function=OpenAIEmbeddings(openai_api_key=openAI_API_Key)

In [28]:
from langchain.vectorstores import Chroma

In [29]:
db=Chroma.from_documents(documents=docs, embedding=my_embedding_function, persist_directory='./some_new_mkultra')

In [30]:
db.persist()

In [32]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.chat_models import ChatOpenAI

Now we would ask a question. We want to consider this question and finds out what is the other variation of this question according to the embedded text

In [34]:
question='when was this classified?'

In [43]:
llm=ChatOpenAI(openai_api_key=openAI_API_Key, temperature=0)

note: we can change the temperature

In [44]:
my_retriever= MultiQueryRetriever.from_llm(db.as_retriever(),llm=llm)

In [45]:
unique_doc=my_retriever.get_relevant_documents(query=question)

Note: the upper code doesnot answer the query this just returns the most relavant text in the documents

Now we have all the texts which are related to questions directly and indirectly

In [46]:
len(unique_doc)

7

In [47]:
unique_doc[0]

Document(page_content="MKNAOMI is the code name for a joint Department of Defense/CIA research program from the 1950s through to the 1970s. Unclassified information about the MKNAOMI program and the related Special Operations Division is scarce. It is generally reported to be a successor to the MKULTRA project focusing on biological projects including biological warfare agents—specifically, to store materials that could either incapacitate or kill a test subject and to develop devices for the diffusion of such materials.During its first twenty years, the CIA engaged in projects designed to increase U.S. biological and chemical warfare capabilities. Project MKNAOMI was initiated to provide the CIA with a covert support base to meet its top-secret operational requirements. The goal was to have a robust arsenal of lethal and incapacitating materials within the CIA's Technical Services Division (TSD). This would enable the TSD to serve as a center for supplying biological and chemical mate